# Basic example for the DASF Framework

_Philipp S. Sommer_

Supplementary notebook to live demo at the Data Science Symposium No. 7.

## A very basic backend module

Take a netCDF variable and computes the sum.

In [1]:
cat > backend.py << EOF

from demessaging import main
from demessaging.types.xarray import DataArray


def compute_sum(da: DataArray) -> DataArray:
    """Compute the sum over a data array.

    Parameters
    ----------
    da : DataArray
        The input data array

    Returns
    -------
    DataArray
        The sum of the data array
    """
    return da.sum()


if __name__ == "__main__":
    main(messaging_config=dict(topic="hello_world"), members=["compute_sum"])
    
EOF

: 1

## Command line utility

The `main` function we used above gives us a command line utility

In [2]:
python backend.py --help

usage: backend.py [-h] [-t MESSAGING_CONFIG.TOPIC] [-m MODULE_NAME] [-d DOC]
                  [-H PULSAR_CONFIG.HOST] [-p PULSAR_CONFIG.PORT]
                  [--persistent PULSAR_CONFIG.PERSISTENT]
                  [--tenant PULSAR_CONFIG.TENANT]
                  [--namespace PULSAR_CONFIG.NAMESPACE]
                  [--max_workers MESSAGING_CONFIG.MAX_WORKERS]
                  [--queue_size MESSAGING_CONFIG.QUEUE_SIZE]
                  [--max_payload_size MESSAGING_CONFIG.MAX_PAYLOAD_SIZE]
                  [--websocket-url WEBSOCKETURL_CONFIG.WEBSOCKET_URL]
                  [--members member [member ...]]
                  {test-connect,listen,schema,send-request,shell,generate} ...

optional arguments:
  -h, --help            show this help message and exit
  -m MODULE_NAME, --module MODULE_NAME
                        Name of the backend module. Default: __main__
  -d DOC, --description DOC
                        The documentation of the object. If empty, this will
      

: 1

## Command line utility

We can use it to connect to the Apache pulsar

In [3]:
python backend.py listen &

[1] 17891


: 1

and to generate the client stub

In [4]:
python backend.py generate

connection attempt 1
unable to establish connection (reason: Connection is already closed.), next attempt in 10 sec.
"""2004l

"""
from typing import Callable

import demessaging.types.xarray
from demessaging import BackendModule as _BackendModule
from demessaging import main
from demessaging.config import ModuleConfig

NoneType = type(None)


__all__ = ["compute_sum"]


def compute_sum(
    da: demessaging.types.xarray.DataArray,
) -> demessaging.types.xarray.DataArray:
    """
    Compute the sum over a data array.

    Parameters
    ----------
    da : DataArray
        The input data array

    Returns
    -------
    DataArray
        The sum of the data array
    """
    request = {"func_name": "compute_sum", "da": da}

    model = BackendModule.parse_obj(request)
    response = model.compute()

    return response.__root__  # type: ignore


backend_config = ModuleConfig.parse_raw(
    """
{
    "messaging_config": {
        "topic": "hello_world",
        "max_workers": null,
 

: 1

## Command line utility

If we pipe this into a file, we generate a module that we can import.

In [5]:
python backend.py generate > api.py

: 1

## Let's try it

We defined compute_sum, to take xarray DataArray.

```python
def compute_sum(da: DataArray) -> DataArray:
    """Compute the sum over a data array.

    Parameters
    ----------
    da : DataArray
        The input data array

    Returns
    -------
    DataArray
        The sum of the data array
    """
    return da.sum()
```

In [6]:
python -- << EOF

import xarray as xr
from api import compute_sum

summed = compute_sum(xr.DataArray([1, 2, 3]))

print('------')
print(summed)

EOF

connection attempt 2
connection to ws://localhost:8080/ws/v2/consumer/non-persistent/public/default/hello_world/backend-module-2022-06-27T09:19:54 established
waiting for incoming request
connection to ws://localhost:8080/ws/v2/producer/non-persistent/public/default/hello_world established
connection to ws://localhost:8080/ws/v2/consumer/non-persistent/public/default/hello_world_cBjawdLU/python-backend-2022-06-27T09:20:14.634672 established
[2022-06-27 09:20:14.707167] processing request
connection to ws://localhost:8080/ws/v2/producer/non-persistent/public/default/hello_world_cBjawdLU established
request successful
------
<xarray.DataArray ()>
array(6, dtype=int32)


: 1

## Let's try it

We can also load netCDF files from the disk and send them via the web

In [7]:
python -- << EOF

import xarray as xr
from api import compute_sum

ds = xr.open_dataset('demo.nc')
summed = compute_sum(ds.t2m[0])

print('------')
print(summed)

EOF

connection to ws://localhost:8080/ws/v2/producer/non-persistent/public/default/hello_world established
connection to ws://localhost:8080/ws/v2/consumer/non-persistent/public/default/hello_world_BpLwTSUw/python-backend-2022-06-27T09:20:22.089103 established
[2022-06-27 09:20:22.183003] processing request
connection to ws://localhost:8080/ws/v2/producer/non-persistent/public/default/hello_world_BpLwTSUw established
request successful
------
<xarray.DataArray 't2m' ()>
array(5142920., dtype=float32)
Coordinates:
    lev      float64 ...
    time     datetime64[ns] ...


: 1